In [1]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 
from PIL import Image
from torch.utils.data import DataLoader,Dataset,random_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from os import listdir
from sklearn.model_selection import train_test_split
import pandas as pd
from os.path import isfile, join

In [6]:
t.cuda.set_device(8)

device = t.device('cuda' if t.cuda.is_available() else 'cpu')

In [7]:
audio_dir = "/raid/amana/lavish_multi_model/emotion_detection/data/raw_audio/"
image_dir = "/raid/amana/lavish_multi_model/emotion_detection/data/video_frames/"

In [8]:
import torchvision
import glob


def get_image_from_file(image_dir,file_name):
    my_normalize = Compose([
				Resize([224,224], interpolation=Image.BICUBIC),
				Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
			])
    total_num_frames = len(glob.glob(image_dir+'/'+file_name+'/*.jpg'))
    sample_indx = np.linspace(1, total_num_frames , num=10, dtype=int)
    total_img = []
    for vis_idx in range(10):
        tmp_idx = sample_indx[vis_idx]
        tmp_img = torchvision.io.read_image(image_dir+'/'+file_name+'/'+ str("{:08d}".format(tmp_idx))+ '.jpg')/255
        tmp_img = my_normalize(tmp_img)
        total_img.append(tmp_img)
    total_img = t.stack(total_img)
    total_img = t.mean(total_img,dim=0)
    return total_img

In [9]:
from transformers import AutoImageProcessor, ViTModel
import torch
from datasets import load_dataset


image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)

In [10]:
def normalize_vector(vector):
    vector = vector.detach().numpy()
    vector_min = np.min(vector)
    vector_max = np.max(vector)
    normalized_vector = (vector - vector_min) / (vector_max - vector_min)
    return t.tensor(normalized_vector)

def get_dataset_from_clip(image_folder,file_name,device):
    input = get_image_from_file(image_folder,file_name)
    input = normalize_vector(input)
    inputs = image_processor(images=input, return_tensors="pt").to(device)
    outputs = model(**inputs)
    pooled_output = outputs.last_hidden_state
    return pooled_output.squeeze(0).cpu().detach().numpy()

In [11]:
onlyfiles = [f for f in listdir(image_dir)]
data = pd.read_csv("/raid/amana/lavish_multi_model/emotion_detection/data/text_data.csv")
data['filename'] = [f'dia{a}_utt{b}' for a,b in zip(data['Dialogue_ID'],data['Utterance_ID'])]
labels = list(data['Emotion'])

In [13]:
dataset = []

for a,b in tqdm(zip(onlyfiles,labels)):
    x = get_dataset_from_clip(image_dir,a,device)
    dataset.append([x,b])

0it [00:00, ?it/s]/tmp/ipykernel_50094/648332381.py:7: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  Resize([224,224], interpolation=Image.BICUBIC),
535it [30:27,  3.42s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 dataset = []                                                                                 │
│   2                                                                                              │
│   3 for a,b in tqdm(zip(onlyfiles,labels)):                                                      │
│ ❱ 4 │   x = get_dataset_from_clip(image_dir,a,device)                                            │
│   5 │   dataset.append([x,b])                                                                    │
│   6                                                                                              │
│                                                                                                  │
│ in get_dataset_from_clip:9                                                                       │
│                                                                                                  │
│    6 │   return t.tensor(normalized_vector)                                                      │
│    7                                                                                             │
│    8 def get_dataset_from_clip(image_folder,file_name,device):                                   │
│ ❱  9 │   input = get_image_from_file(image_folder,file_name)                                     │
│   10 │   input = normalize_vector(input)                                                         │
│   11 │   inputs = image_processor(images=input, return_tensors="pt").to(device)                  │
│   12 │   outputs = model(**inputs)                                                               │
│                                                                                                  │
│ in get_image_from_file:16                                                                        │
│                                                                                                  │
│   13 │   for vis_idx in range(10):                                                               │
│   14 │   │   tmp_idx = sample_indx[vis_idx]                                                      │
│   15 │   │   tmp_img = torchvision.io.read_image(image_dir+'/'+file_name+'/'+ str("{:08d}".fo    │
│ ❱ 16 │   │   tmp_img = my_normalize(tmp_img)                                                     │
│   17 │   │   total_img.append(tmp_img)                                                           │
│   18 │   total_img = t.stack(total_img)                                                          │
│   19 │   total_img = t.mean(total_img,dim=0)                                                     │
│                                                                                                  │
│ /raid/amana/miniconda3/envs/lamol/lib/python3.8/site-packages/torchvision/transforms/transforms. │
│ py:60 in __call__                                                                                │
│                                                                                                  │
│     57 │                                                                                         │
│     58 │   def __call__(self, img):                                                              │
│     59 │   │   for t in self.transforms:                                                         │
│ ❱   60 │   │   │   img = t(img)                                                                  │
│     61 │   │   return img                                                                        │
│     62 │                                                                                         │
│     63 │   def __repr__(self):                             

In [40]:
emotion_label = {'anger' : 1, 'disgust' : 2, 'fear' : 3,
                 'joy' : 4, 'neutral' : 5, 'sadness' : 6, 'surprise' : 0}

In [41]:
dataset1 = np.array([np.array([a[0],emotion_label[a[1]]]) for a in dataset])

In [43]:
np.save("dataset_clip_image_vectors1.npy",dataset1)

In [44]:
X_train,X_test,Y_train,Y_test = train_test_split(dataset1[:,0],dataset1[:,1],test_size=0.2,random_state=42)

In [46]:
import torch.nn as nn
import torch.nn.functional as F

class EmotionMLP(nn.Module):
    def softmax(self,x):
        e_x = t.exp(x - t.max(x))
        return e_x / e_x.sum()
    
    def __init__(self, num_classes):
        super(EmotionMLP, self).__init__()
        self.mat = nn.Parameter(t.rand(50,1))
        self.fc0 = nn.Linear(768,64)
        self.fc4 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = t.relu(t.matmul(x.t(),self.mat))
        x = x.view(-1, 768)
        x = t.relu(self.fc0(x))
        x = t.relu(self.fc4(x))
        return x

In [47]:
import warnings
warnings.filterwarnings("ignore")
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

In [48]:
num_classes = 7
model = EmotionMLP(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = t.optim.Adam(model.parameters(), lr=0.0001)

In [49]:
def train(model,X_train,Y_train,optimizer,criterion,num_epochs,device):
    loss_arr = []
    for epoch in tqdm(range(num_epochs)):
        # model.train()
        total_loss = 0
        for x,y in zip(X_train,Y_train):
            x = t.tensor(x)
            # Forward pass
            outputs = model(x.to(device))
            loss = criterion(outputs,t.tensor([y]).to(device)).to(device)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        loss_arr.append(total_loss/len(X_train))
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {total_loss/len(X_train)}')
    return loss_arr

num_epochs = 20
lossarr = train(model,X_train,Y_train,optimizer,criterion,num_epochs,device)

  5%|▌         | 1/20 [00:07<02:31,  7.97s/it]

Epoch 1/20, Training Loss: 1.7472953045448831


 10%|█         | 2/20 [00:17<02:35,  8.66s/it]

Epoch 2/20, Training Loss: 1.7368239916970254


 15%|█▌        | 3/20 [00:25<02:25,  8.56s/it]

...

Epoch 18/20, Training Loss: 1.7145708174420586


 95%|█████████▌| 19/20 [03:23<00:22, 22.86s/it]

Epoch 19/20, Training Loss: 1.7142838453359508


100%|██████████| 20/20 [03:38<00:00, 10.94s/it]

Epoch 20/20, Training Loss: 1.7139769481429925


In [52]:
def test(model,X_test,Y_test,device):
    acc = 0
    def accuracy(y_true, y_pred):
        eq = t.eq(y_true, y_pred).int()
        return sum(eq)/len(eq)

    with t.no_grad():
        model.eval()
        for x,y in zip(X_test,Y_test):
            outputs = model(t.tensor(x).to(device))
            outputs1 = outputs.detach().cpu()
            outputs1 = outputs1.argmax(dim=0)
            acc += accuracy(t.tensor([y]),outputs1)
        print(f"accuracy: {(acc/len(X_test))*100: 0.2f}%")
        
test(model,X_test,Y_test,device)

accuracy:  12.61%
